In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport Compute_Jacobian_MA
@pyimport GLS_Apr_weekday_PM_ext

In [2]:
numNodes = Compute_Jacobian_MA.numNodes;
numLinks = Compute_Jacobian_MA.numLinks;
numODpairs = Compute_Jacobian_MA.numODpairs;
numZones = Compute_Jacobian_MA.numZones
od_pairs = Compute_Jacobian_MA.od_pairs;
link_list_js = Compute_Jacobian_MA.link_list_js;
link_length_list = Compute_Jacobian_MA.link_length_list;

flow_observ = GLS_Apr_weekday_PM_ext.x_

74×30 Array{Float64,2}:
 1366.35    791.95    771.324  …  1015.01        1624.03     280.272
 1206.76    992.338   991.158     1000.59        1273.13     570.559
 2114.24   1499.68   1200.6       1171.78        1608.83     947.273
 2273.84   1299.3     980.768     1186.21        1959.73     656.986
   58.465   201.97    271.021       86.5748        40.0646   261.282
  139.064   247.675   201.826  …    23.5801       146.69     143.578
 1193.7    1571.81   1353.56       781.132       1109.21    1362.48 
  953.514  1726.49   1642.59       829.701        651.679   1770.47 
 2124.9    1398.79    938.436     1533.92        1629.54    1007.64 
 2365.09   1244.1     649.406     1485.35        2087.07     599.646
  406.406   514.047   567.011  …   469.666        455.737    655.187
  281.997  1677.2    1591.06       204.042        239.965   1660.44 
  204.522   246.959  1020.38         1.2451e-13   268.26    1131.08 
    ⋮                          ⋱                                    
  817.703 

In [3]:
include("../Julia_files/initia_data.jl");
include("prepare_data_MA.jl");
include("extract_data_MA.jl");
include("../Julia_files/tap_MSA.jl");
include("../Julia_files/inverseVI.jl");
include("../Julia_files/demands_adjustment_gradi.jl");

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in readall(::String, ::Vararg{String,N}) at ./deprecated.jl:30
 in furInfo() at /home/jzh/Research/Traffic/InverseVIsTraffic/08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_PM/prepare_data_MA.jl:183
 in include_from_node1(::String) at ./loading.jl:488
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/jzh/.julia/v0.5/IJulia/src/execute_request.jl:169
 in eventloop(::ZMQ.Socket) at /home/jzh/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##9#15)() at ./task.jl:360
while loading /home/jzh/Research/Traffic/InverseVIsTraffic/08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_PM/extract_data_MA.jl, in expression starting on line 10
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in readall(::String, ::Vararg{String,N}) at ./deprecated.jl:30
 in furInfo() at /home/jzh/Research/Traffic/InverseVIsTraffic/08_develop_new_OD_demand_estimator_MA

In [4]:
demandsDict

Dict{Any,Any} with 2 entries:
  0 => Dict{Any,Any}(Pair{Any,Any}((6,9),0.0),Pair{Any,Any}((18,3),0.0),Pair{An…
  1 => Dict{Any,Any}(Pair{Any,Any}((6,9),0.0),Pair{Any,Any}((18,3),0.0),Pair{An…

In [5]:
demandsDiffDict[1] = norm(demandsDicToVec(demandsDict[1]) - demandsDicToVec(demandsDict[0]))/
                     norm(demandsDicToVec(demandsDict[0]));

In [6]:
demandsDiffDict[1]

0.0

In [7]:
gamma1 = 1
gamma2 = 1

1

In [8]:
function demandsDictFixed(day)
#     day = 4  # day of April

    # observed flow vector
    xl = flow_observ[:, day]

    tapFlows = Dict()

    for i = 1:length(ta_data.start_node)
        key = (ta_data.start_node[i], ta_data.end_node[i])
        tapFlows[key] = xl[i]
    end

    tapFlowVect = xl;

    # get observed flow vector (corresponding to ground truth demands and ground truth costs)
    tapFlowDicDict[0], tapFlowVecDict[0] = tapFlows, tapFlowVect;

    # get arcs data corresponding to ground truth demands and flows
    arcsDict[0] = observFlow("./benchmark_data/MA_net.txt", tapFlowDicDict[0]);

    arcsDict[0]

    deg = 8
    c = 0.5
    lam = 10000.0

    coeffs_dict_Apr_PM_ = readall("../temp_files/coeffs_dict_Apr_PM.json")
    coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
    fcoeffs = coeffs_dict_Apr_PM_["($(deg),$(c),$(lam),1)"]

    # fcoeffs = [1, 0, 0, .15]

    demandsVecDict[1] = demandsDicToVec(demandsDict[1]);
    objFunDict[1] = objF(gamma1, gamma2, demandsVecDict[1], demandsVecDict[1], fcoeffs);
    
    # get initial flow vector (corresponding to initial demands)
    tapFlowDicDict[1], tapFlowVecDict[1] = tapMSA(demandsDict[1], fcoeffs);

    demandsVecDict[0] = demandsDicToVec(demandsDict[0]);

    # get arcs data corresponding to initial demands and flows
    arcsDict[1] = observFlow("./benchmark_data/MA_net.txt", tapFlowDicDict[1]);

    linkCostDicDict[1] = tapFlowVecToLinkCostDict(tapFlowVecDict[1], fcoeffs);

    linkCostDicDict[1]["0"], link_length_list[1]

    jacobiSpiessDict[1] = Compute_Jacobian_MA.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, 
                                                  link_list_js, [linkCostDicDict[1]["$(i)"] for i=0:numLinks-1]);

    # maximum number of iterations
    N = 10;  

    # Armijo rule parameters
    rho = 2;
    M = 10;

    # search direction parameter
    epsilon_1 = 0;

    # stop criterion parameter
    epsilon_2 = 1e-20;

    for l = 1:N

        jacobDict[l] = jacobiSpiessDict[l]
        
        demandsVecDict[l] = demandsDicToVec(demandsDict[l]);
        
        descDirecDict[l] = descDirec(gamma1, gamma2, demandsVecDict[l], demandsVecDict[1], tapFlowVecDict[l], tapFlowVecDict[0], 
            jacobDict[l], numODpairs, numLinks);     

        searchDirecDict[l] = searchDirec(demandsVecDict[l], descDirecDict[l], epsilon_1);

        thetaMaxDict[l] = thetaMax(demandsVecDict[l], searchDirecDict[l]);

        demandsVecDict[l+1] = similar(demandsVecDict[0]);

        demandsVecDict[l+1], objFunDict[l+1] = armijo(gamma1, gamma2, objFunDict[l], demandsVecDict[l], 
            demandsVecDict[0], fcoeffs, searchDirecDict[l], thetaMaxDict[l], rho, M);

        demandsDict[l+1] = demandsVecToDic(demandsVecDict[l+1]);

        tapFlowDicDict[l+1], tapFlowVecDict[l+1] = tapMSA(demandsDict[l+1], fcoeffs);

        arcsDict[l+1] = observFlow("./benchmark_data/MA_net.txt", tapFlowDicDict[l+1]);

        linkCostDicDict[l+1] = tapFlowVecToLinkCostDict(tapFlowVecDict[l+1], fcoeffs);

        jacobiSpiessDict[l+1] = Compute_Jacobian_MA.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, 
                                                  link_list_js, [linkCostDicDict[l+1]["$(i)"] for i=0:numLinks-1]);

        demandsDiffDict[l+1] = norm(demandsVecDict[l+1] - demandsVecDict[0]) / norm(demandsVecDict[0]);

        # stopping criterion
        if (objFunDict[l] - objFunDict[l+1]) / objFunDict[1] < epsilon_2
            break
        end

    end

    # normalize objective function value
    for l = 1:(length(objFunDict))
        norObjFunDict[l] = objFunDict[l] / objFunDict[1];
    end


    outfile = open("./results/demandsVecDict.json", "w")

    JSON.print(outfile, demandsVecDict)

    close(outfile)

    outfile = open("./results/demandsDict$(day).json", "w")

    JSON.print(outfile, demandsDict)

    close(outfile)

    outfile = open("./results/tapFlowDicDict.json", "w")

    JSON.print(outfile, tapFlowDicDict)

    close(outfile)

    outfile = open("./results/tapFlowVecDict.json", "w")

    JSON.print(outfile, tapFlowVecDict)

    close(outfile)

    demandsDict[length(demandsDict)-1]

    demandsDict_ = Dict{}()

    for key in keys(demandsDict[length(demandsDict)-1])
        demandsDict_[key] = demandsDict[length(demandsDict)-1][key] 
    end

    outfile = open("./results/demandsDictFixed$(day).json", "w")

    JSON.print(outfile, demandsDict_)

    close(outfile)
end

demandsDictFixed (generic function with 1 method)

In [9]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]
# week_day_Apr_list = [5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]

for day in week_day_Apr_list
    demandsDictFixed(day)
end

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in readall(::String, ::Vararg{String,N}) at ./deprecated.jl:30
 in demandsDictFixed(::Int64) at ./In[8]:28
 in macro expansion; at ./In[9]:5 [inlined]
 in anonymous at ./<missing>:?
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/jzh/.julia/v0.5/IJulia/src/execute_request.jl:169
 in eventloop(::ZMQ.Socket) at /home/jzh/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##9#15)() at ./task.jl:360
while loading In[9], in expression starting on line 4
